In [5]:
# nhits_pipeline.py
import os
import re
import glob
import pandas as pd
import numpy as np
import optuna
import torch

from darts import TimeSeries
from darts.models import NHiTSModel
from darts.metrics import mae, mape, rmse
from darts.utils.likelihood_models import GaussianLikelihood
from darts.utils.timeseries_generation import datetime_attribute_timeseries
from darts.dataprocessing.transformers import Scaler

In [6]:


# --- User settings ---
PRODUCTS_DIR = "products_csv"         # folder with product CSVs
OUTPUT_DIR = "nhits_models"           # folder to save per-product models
COMMON_MODEL_PATH = "common_nhits_model.h5"  # final shared model save path
N_TRIALS = 20                         # Optuna trials per product (as requested)
HORIZON = 6                           # forecast horizon (months) to optimize for
MAX_EPOCHS = 80                       # training epochs (you can reduce for speed)
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# --- Helpers ---
def safe_name(s):
    return re.sub(r'[^A-Za-z0-9]+', '_', s).strip('_')

def load_product_series(csv_path):
    df = pd.read_csv(csv_path)
    # Expect columns: date, product, quantity
    df['date'] = pd.to_datetime(df['date'])
    df = df.sort_values('date')
    # set monthly frequency (if monthly data)
    ts = TimeSeries.from_dataframe(df, 'date', 'quantity')
    return ts, df['product'].iloc[0]  # series, product_name

# --- Optuna objective for a single series ---
def make_optuna_objective(train_ts, val_ts, input_chunk_hint=24, horizon=HORIZON):
    """
    Returns an objective(trial) that builds a NHiTS model with trial hyperparams,
    fits it and returns validation MAE.
    """
    def objective(trial):
        # sample hyperparameters
        # Adjusted range for input_chunk_len to ensure min value is >= 24 if possible
        min_icl = max(6, min(24, input_chunk_hint // 2))
        max_icl = max(24, input_chunk_hint)
        input_chunk_len = trial.suggest_int("input_chunk_len", min_icl, max_icl)
        num_blocks = trial.suggest_int("num_blocks", 1, 4)
        num_layers = trial.suggest_int("num_layers", 1, 3)
        layer_size = trial.suggest_int("layer_size", 32, 512, log=True)
        dropout = trial.suggest_float("dropout", 0.0, 0.3)
        lr = trial.suggest_loguniform("lr", 1e-4, 1e-2)
        batch_size = trial.suggest_categorical("batch_size", [16, 32, 64])

        model = NHiTSModel(
            input_chunk_length = input_chunk_len,
            output_chunk_length = horizon,
            n_epochs = 20,                          # quick training inside optuna trials
            batch_size = batch_size,
            num_blocks = num_blocks,
            num_layers = num_layers,
            layer_widths = [layer_size],            # NHiTS expects widths as list
            dropout = dropout,
            optimizer_kwargs = {"lr": lr},
            random_state = 42,
            likelihood = None,
            pl_trainer_kwargs = {"accelerator": "gpu"} if DEVICE.startswith("cuda") else {}
        )

        try:
            model.fit(train_ts, verbose=False)
            # Forecast on validation set (one-shot)
            pred = model.predict(n=horizon)
            # Align index length if necessary: compute metrics using last horizon points
            # We'll compute MAE between val_ts[:horizon] and pred
            # If val_ts length < horizon, use full length
            eval_len = min(len(val_ts), len(pred))
            val_sub = val_ts[-eval_len:]
            pred_sub = pred[:eval_len]
            score = mae(val_sub, pred_sub)
        except Exception as e:
            # If training fails, return a large loss
            print("Trial failed:", e)
            score = 1e9
        # free GPU memory
        del model
        torch.cuda.empty_cache()
        return float(score)
    return objective

# --- Per-product tuning / training function ---
def tune_and_train_product(ts: TimeSeries, product_name: str):
    # Train/validation split: last HORIZON months for validation
    if len(ts) <= HORIZON + 6:
        # very short - use 80/20 split in time
        split = int(len(ts) * 0.8)
        train_ts, val_ts = ts[:split], ts[split:]
    else:
        train_ts, val_ts = ts[:-HORIZON], ts[-HORIZON:]

    # Scale series
    scaler = Scaler()
    train_scaled = scaler.fit_transform(train_ts)
    val_scaled = scaler.transform(val_ts)

    # Make Optuna study
    study = optuna.create_study(direction="minimize", sampler=optuna.samplers.TPESampler(seed=42))
    # input_chunk_hint for Optuna is now better constrained
    input_chunk_hint = min(36, max(12, len(train_ts)//2))
    objective = make_optuna_objective(train_scaled, val_scaled, input_chunk_hint=input_chunk_hint)
    study.optimize(objective, n_trials=N_TRIALS, show_progress_bar=True)

    print(f"Best params for {product_name}: {study.best_params}, best_val_mae={study.best_value}")

    # Build final model with best params and longer epochs on full train+val
    best = study.best_params
    final_model = NHiTSModel(
        input_chunk_length = best.get("input_chunk_len", 24),
        output_chunk_length = HORIZON,
        n_epochs = MAX_EPOCHS,
        batch_size = best.get("batch_size", 32),
        num_blocks = best.get("num_blocks", 2),
        num_layers = best.get("num_layers", 2),
        layer_widths = [best.get("layer_size", 128)],
        dropout = best.get("dropout", 0.0),
        optimizer_kwargs = {"lr": best.get("lr", 1e-3)},
        random_state = 42,
        pl_trainer_kwargs = {"accelerator": "gpu"} if DEVICE.startswith("cuda") else {}
    )

    # Fit on the entire series (train+val)
    scaled_full = scaler.fit_transform(ts)
    final_model.fit(scaled_full, verbose=True)

    # Save model and scaler
    filename = os.path.join(OUTPUT_DIR, safe_name(product_name) + "_nhits.h5")
    final_model.save(filename)
    # Save scaler object for later inverse transform
    scaler_path = os.path.join(OUTPUT_DIR, safe_name(product_name) + "_scaler.pkl")
    pd.to_pickle(scaler, scaler_path)

    print(f"Saved model: {filename} and scaler: {scaler_path}")
    return final_model, scaler


In [ ]:

# CRITICAL FIX: global_model.predict() returns a list of TimeSeries when trained on a list
# We must call it once and iterate through the results
print("Generating global forecasts...")
all_common_preds = global_model.predict(n=FORECAST_H)
# Ensure the number of forecasts matches the number of products
if len(all_common_preds) != len(product_names):
    print("Warning: Number of global forecasts does not match number of products.")

print("Generating ensemble forecasts...")
# Use enumerate to get the index, which corresponds to the global forecast index
for i, (prod_name, ts_scaled) in enumerate(zip(product_names, product_series_list)):
    # per-product model file load
    per_product_path = os.path.join(OUTPUT_DIR, safe_name(prod_name) + "_nhits.h5")
    try:
        # Load per-product model
        per_model = NHiTSModel.load(per_product_path)
        # Get the corresponding common forecast for this product
        common_pred = all_common_preds[i]

    except Exception as e:
        print(f"Failed loading per-product model or accessing global forecast for {prod_name}. Skipping ensemble: {e}")
        continue

    # Generate per-product forecast
    per_pred = per_model.predict(n=FORECAST_H)
    
    # average forecasts (ensure matching lengths)
    # inverse transform using per-product scaler
    # load scaler
    scaler_path = os.path.join(OUTPUT_DIR, safe_name(prod_name) + "_scaler.pkl")
    scaler = pd.read_pickle(scaler_path)
    
    # both per_pred and common_pred are scaled series; inverse transform using scaler
    per_inv = scaler.inverse_transform(per_pred)
    common_inv = scaler.inverse_transform(common_pred)

    # convert to pandas and average
    per_df = per_inv.pd_dataframe().rename(columns={per_inv.pd_dataframe().columns[0]:"per_product"})
    com_df = common_inv.pd_dataframe().rename(columns={common_inv.pd_dataframe().columns[0]:"common"})
    merged = pd.concat([per_df, com_df], axis=1) # fillna(method='ffill') is not usually needed for forecasts of same length
    merged['ensemble'] = merged.mean(axis=1)

    outpath = os.path.join(FORECAST_OUTPUT, safe_name(prod_name) + "_forecast.csv")
    merged.to_csv(outpath, index=True)
    print("Saved forecast:", outpath)

print("Done.")